In [1]:
""" Type hierarchy for abstract linear operators
"""
class DaskLinearOperator(object):
    """ Base class for abstract :obj:`dask.array.Array`-based linear operators 
    """
    def __init__(self, data, name=None):
        self._data = data
        self._name = 'DLO-' + data.name if name is None else str(name)

    @property
    def data(self):
        return self._data

    @property
    def name(self):
        return self._name

    @property
    def shape(self):
        return self._data.shape

    @property
    def size(self):
        return self._data.size

    @property
    def chunks(self):
        return self._data.chunks

    @property
    def numblocks(self):
        return self._data.numblocks

    @property
    def dtype(self):
        return self._data.dtype

In [2]:
import dask.array as da
A = da.random.random((100, 50), chunks=20)
Adlo = DaskLinearOperator(A)

In [3]:
assert Adlo.size == A.size
assert Adlo.shape == A.shape
assert Adlo.chunks == A.chunks
assert Adlo.numblocks == A.numblocks
assert Adlo.dtype == A.dtype


In [4]:
class DLOSymmetric(DaskLinearOperator):
    def __init__(self, data, name=None):
        name = 'DLO-symmetric-' + data.name if name is None else name
        DaskLinearOperator.__init__(self, data, name=name)
        assert self.shape[0] == self.shape[0]
        assert self.chunks[0] == self.chunks[1]

In [5]:
try:
    DLOSymmetric(A)
except AssertionError:
    print 'fail on dims'

try:
    DLOSymmetric(da.random.random((100, 100), chunks=(10,20)))
except AssertionError:
    print 'fail on chunks'

Asymm = DLOSymmetric(da.random.random((100, 100), chunks=10))
assert Asymm.numblocks == (10, 10)

fail on dims
fail on chunks


In [6]:
class DLODense(DaskLinearOperator):
    def __init__(self, data, name=None):
        name = 'DLO-dense-' + data.name if name is None else name
        DaskLinearOperator.__init__(self, data, name=name)

In [7]:
Adn = DLODense(A)
assert Adn.numblocks == A.numblocks

In [8]:
class DLODiagonal(DLOSymmetric):
    def __init__(self, data, name=None):
        name = 'DLO-diagonal-' + data.name if name is None else name
        DLOSymmetric.__init__(self, data, name=name)

    @property
    def shape(self):
        return self._data.shape[0], self._data.shape[0]

    @property
    def chunks(self):
        return self._data.chunks[0], self._data.chunks[0]

    @property
    def numblocks(self):
        return self._data.numblocks[0], self._data.numblocks[0]

In [9]:
Adn = DLODiagonal(da.diag(da.random.random((100, 100), chunks=50)))
assert Adn.numblocks == (2, 2)
assert Adn.data.numblocks == (2,)

In [10]:
class DLOGram(DLOSymmetric):
    def __init__(self, data, name=None):
        self.transpose = data.shape[0] < data.shape[1]
        self._idx = 1 - int(self.transpose)
        name = 'DLO-gram-' + data.name if name is None else name
        DLOSymmetric.__init__(self, data, name=name)

    @property
    def shape(self):
        return self._data.shape[self._idx], self._data.shape[self._idx]

    @property
    def chunks(self):
        return self._data.chunks[self._idx], self._data.chunks[self._idx]

    @property
    def numblocks(self):
        return self._data.numblocks[self._idx], self._data.numblocks[self._idx]

In [11]:
Agm = DLOGram(A)
assert Agm.numblocks == (A.numblocks[1], A.numblocks[1])
Agm2 = DLOGram(da.transpose(A))
assert Agm.shape == Agm2.shape

In [12]:
class DLORegularizedGram(DLOGram):
    def __init__(self, data, name=None, regularization=1.):
        self.regularization = float(regularization)
        name = 'DLO-regularized-gram-' + data.name if name is None else name
        DLOGram.__init__(self, data, name=name)

In [13]:
Agm = DLORegularizedGram(A)
assert Agm.regularization == 1